TRAINING ON GOOGLE COLAB WITH GPU AND CUDA

MOUNT DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1.Clone darknet repository in KFS directory


In [ ]:
# Create the directory named darknet
!mkdir -p /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet
#Navigate to directory
%cd /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet
#Clone the repository
!git clone https://github.com/AlexeyAB/darknet

2.Using GPU with CUDA for training

In [ ]:
%cd /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet
# Change OPENCV to 0
!sed -i 's/^OPENCV=.*/OPENCV=1/' Makefile
# Change GPU to 0
!sed -i 's/^GPU=.*/GPU=1/' Makefile
# Change CUDNN to 0
!sed -i 's/^CUDNN=.*/CUDNN=1/' Makefile
# Change CUDNN_HALF to 0
!sed -i 's/^CUDNN_HALF=.*/CUDNN_HALF=1/' Makefile
# Change LIBSO to 0
!sed -i 's/^LIBSO=.*/LIBSO=1/' Makefile

3.Clone https://github.com/techzizou/yolov4-tiny-custom_Training and pre-trained weights https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29                                                  

4.obj.names-                                                                                                      
            product

5.obj.data-                                                                                                        
            classes = 1                                                                                            
            train = train.txt                                                                                      
            valid = test.txt                                                                                       
            names = obj.names                                                                                      
            backup= training/   

6.Custom config file
*   change line batch to batch=64
*   change line subdivisions to subdivisions=16
*   change line max_batches to (classes*2000, but not less than number of training images and not less than 6000)
*   change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
*   set network size width=416 height=416 or any value multiple of 32
*   change line classes=1 to your number of objects in each of 2 [yolo]-layers
*   change filters to filters=(classes + 5)x3= 18 in the 2 [convolutional] before each [yolo] layer

7.Build darknet.

In [ ]:
%cd /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet
!make

8.Verify Darknet build and copy files to data directory

In [ ]:
# Verify the build by checking the darknet executable
!ls -l darknet

# Change permissions to make darknet executable
!chmod +x darknet

# Run a simple test to ensure Darknet is working
!./darknet

In [ ]:
# Remove all files except the 'labels' folder
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
!rm -rf obj/ #If obj file does not exist to delete, might throw errow. Reforat code to not delete obj then.
%cd ..
!rm -rf cfg/
!mkdir cfg

In [ ]:
#copy the datasets zip file to the root darknet folder
!cp /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/obj.zip ../

# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/

In [ ]:
# copy the obj.names and obj.data files so that they are now in /darknet/data/ folder
!cp /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/obj.names ./data
!cp /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/obj.data ./data
!cp /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/process.py ./
!cp /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/yolov4-tiny-custom.cfg ./cfg

9.Generate train.txt and test.txt and save in data directory

In [ ]:
import os
import random

# Define the paths
data_path = '/content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet/data/obj'

# Get all files in the directory
files = os.listdir(data_path)

# Separate images and annotations
images = [f for f in files if f.endswith('.png')]
annotations = [f for f in files if f.endswith('.txt')]

# Print the number of images and annotations
print(f"Total images: {len(images)}")
print(f"Total annotations: {len(annotations)}")

# Check for missing annotations
image_files_with_annotations = [f for f in images if f.replace('.png', '.txt') in annotations]
missing_annotations = [f for f in images if f.replace('.png', '.txt') not in annotations]

print(f"Images with annotations: {len(image_files_with_annotations)}")
print(f"Images missing annotations: {len(missing_annotations)}")

# Print some missing annotation files if any
if missing_annotations:
    print("Some images are missing annotation files:")
    for img in missing_annotations[:10]:  # Print the first 10 missing files
        print(img)

# Define the paths
data_path = '/content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet/data/obj'
train_file_path = '/content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet/data/train.txt'
test_file_path = '/content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet/data/test.txt'

# Get all files in the directory
files = os.listdir(data_path)

# Separate images and annotations
images = [f for f in files if f.endswith('.png')]
annotations = [f.replace('.txt', '.png') for f in files if f.endswith('.txt')]

# Filter out images that don't have corresponding annotations
dataset = [os.path.join(data_path, img) for img in images if img in annotations]

# Shuffle the dataset
random.shuffle(dataset)

# Define the split ratio (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(dataset) * split_ratio)

# Split into train and test sets
train_set = dataset[:split_index]
test_set = dataset[split_index:]

# Write train.txt
with open(train_file_path, 'w') as train_file:
    for item in train_set:
        train_file.write(f"{item}\n")

# Write test.txt
with open(test_file_path, 'w') as test_file:
    for item in test_set:
        test_file.write(f"{item}\n")

print(f"Training and testing files created successfully.\nTrain size: {len(train_set)}\nTest size: {len(test_set)}")

# list the contents of data folder to check if the train.txt and test.txt files have been created
!ls data/

10.Train the model

In [ ]:
# Change the working directory
%cd /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/darknet/darknet

# Run the training command
!./darknet detector train /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/obj.data /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/yolov4-tiny-custom.cfg /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/yolov4-tiny.conv.29 -dont_show -map


Restart model training in case of interruption


In [ ]:
#to restart training your custom detector where you left off(using the weights that were saved last)

!./darknet detector train data/obj.data /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/yolov4-tiny-custom.cfg /content/drive/MyDrive/KEY_FRAME_SELECTION/TRAINING/yolov4-tiny/training/yolov4-tiny-custom_last.weights -dont_show